# Link Graph Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
BASE_URL = "https://pta.trunojoyo.ac.id"
START_URL = "https://pta.trunojoyo.ac.id/c_search/byprod/7" # Manajemen
headers = {"User-Agent": "Mozilla/5.0"}

In [4]:
def get_internal_links(url):
    """Ambil semua link internal dari satu halaman"""
    try:
        r = requests.get(url, headers=headers, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")
        links = []
        for a in soup.find_all("a", href=True):
            href = a["href"]
            if href.startswith("/") or href.startswith(BASE_URL):
                if not href.startswith("javascript"):
                    if href.startswith("/"):
                        href = BASE_URL + href
                    links.append(href.split("#")[0])
        return list(set(links))
    except Exception as e:
        print(f"Error akses {url}: {e}")
        return []

def crawl_graph(start_url, max_pages=20):
    visited = set()
    to_visit = [start_url]
    edges = []

    while to_visit and len(visited) < max_pages:
        current = to_visit.pop(0)
        if current in visited:
            continue
        visited.add(current)
        print(f"Crawling: {current}")
        
        links = get_internal_links(current)
        for link in links:
            edges.append({"source": current, "target": link})
            if link not in visited and link not in to_visit:
                to_visit.append(link)
    
    return edges

# Jalankan crawler graph
edges = crawl_graph(START_URL, max_pages=10)

# Simpan ke DataFrame
df = pd.DataFrame(edges)
df.to_excel("pta_graph.xlsx", index=False)

Crawling: https://pta.trunojoyo.ac.id/c_search/byprod/7
Crawling: https://pta.trunojoyo.ac.id/welcome/detail/090211200001
Crawling: https://pta.trunojoyo.ac.id/c_search/byprod/25
Crawling: https://pta.trunojoyo.ac.id/c_search/byprod/7/3
Crawling: https://pta.trunojoyo.ac.id/c_search/byprod/22
Crawling: https://pta.trunojoyo.ac.id/c_search/byprod/8
Crawling: https://pta.trunojoyo.ac.id/c_search/byfac/1
Crawling: https://pta.trunojoyo.ac.id/c_search/byprod/15
Crawling: https://pta.trunojoyo.ac.id/c_search/byprod/99
Crawling: https://pta.trunojoyo.ac.id/c_search/byprod/40


In [5]:
df

,source,target
0,https://pta.trunojoyo.ac.id/c_search/byprod/7,https://pta.trunojoyo.ac.id/welcome/detail/090...
1,https://pta.trunojoyo.ac.id/c_search/byprod/7,https://pta.trunojoyo.ac.id/c_search/byprod/25
2,https://pta.trunojoyo.ac.id/c_search/byprod/7,https://pta.trunojoyo.ac.id/c_search/byprod/7/3
3,https://pta.trunojoyo.ac.id/c_search/byprod/7,https://pta.trunojoyo.ac.id/c_search/byprod/22
4,https://pta.trunojoyo.ac.id/c_search/byprod/7,https://pta.trunojoyo.ac.id/c_search/byprod/8
...,...,...
606,https://pta.trunojoyo.ac.id/c_search/byprod/40,https://pta.trunojoyo.ac.id/c_search/byfac/99
607,https://pta.trunojoyo.ac.id/c_search/byprod/40,https://pta.trunojoyo.ac.id/c_search/byfac/7
608,https://pta.trunojoyo.ac.id/c_search/byprod/40,https://pta.trunojoyo.ac.id/c_search/byprod/10
609,https://pta.trunojoyo.ac.id/c_search/byprod/40,https://pta.trunojoyo.ac.id/c_search/byprod/32
